# Description of the project

It is necessary to decide where to drill a new well. The steps for choosing a location are usually as follows:

- Characteristics for wells are collected in the selected region: oil quality and volume of its reserves;
- Build a model to predict the volume of reserves in new wells;
- Choose the wells with the highest value estimates;
- Determine the region with the maximum total profit of the selected wells.

There are oil samples in three regions. The characteristics for each well in the region are already known. It is necessary to build a model to determine the region where mining will bring the greatest profit by analyzing the possible profit and risks using the Bootstrap technique.

## Conditions of the problem

- Only linear regression is suitable for training the model (the rest are not predictable enough).
- When exploring a region, 500 points are explored, from which the best 200 are selected for profit calculation.
- The budget for the development of wells in the region is 10 billion rubles.
- One barrel of raw materials brings 450 rubles of income. The income from each unit of the product is 450 thousand rubles, since the volume is indicated in thousands of barrels.
- After assessing the risks, you need to leave only those regions in which the probability of losses is less than 2.5%. Among them, choose the region with the highest average profit.

Synthetic data: details of contracts and characteristics of deposits were not disclosed.

# Description of data

Geological exploration data of three regions are in the files:

- `geo_data_0.csv`;
- `geo_data_1.csv`;
- `geo_data_2.csv`;
- `id` — unique identifier of the well;
- `f0`, `f1`, `f2` - three signs of dots (it doesn't matter what they mean, but the signs themselves are significant);
- `product` — volume of reserves in the well (thousand barrels).

# Action plan

1. Download and prepare data
2. Train and test the model for each region: <br />
    2.1. Split the data into training and validation samples in a ratio of 75:25. <br />
    2.2. Train the model and make predictions on the validation set. <br />
    2.3. Store predictions and correct answers on the validation set. <br />
    2.4. Print on the screen the average stock of the predicted raw material and the RMSE of the model. <br />
    2.5. Analyze results. <br />
3. Prepare for profit calculation: <br />
    3.1. Save all key values ​​for calculations in separate variables. <br />
    3.2. Calculate a sufficient volume of raw materials for the break-even development of a new well. Compare the received volume of raw materials with the average stock in each region. <br />
    3.3. Write conclusions on the stage of preparing the calculation of profit. <br />
4. Write a function to calculate profit for selected wells and model predictions: <br />
    4.1. Select wells with maximum prediction values. The number of wells depends on the company's budget and the cost of developing one well. <br />
    4.2. Sum the target value of the volume of raw materials corresponding to these predictions. <br />
    4.3. Calculate the profit for the received volume of raw materials. <br />
5. Calculate risks and profits for each region: <br />
    5.1. Apply the Bootstrap technique with 1000 samples to find the profit distribution. <br />
    5.2. Find the average profit, 95% confidence interval and risk of loss. Loss is negative profit. <br />
    5.3. Write conclusions: suggest a region for well development and justify the choice. <br />

# Import data files, study general information

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from scipy import stats as st

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
PATH = "/datasets"
files = ['geo_data_0.csv', 'geo_data_1.csv', 'geo_data_2.csv']
q = range(len(files))
dfs = []

for file in files:
    with open(os.path.join(PATH, file)) as f:
        dfs.append(pd.read_csv(f, sep=','))

for df in dfs:
    
    display(df)
    df.info()

# Data preparation

Get rid of unnecessary features

In [ ]:
for i in q: dfs[i] = dfs[i].drop(['id'], axis=1)

Divide the samples into sets with features and a target feature

In [ ]:
features = []
target = []

for i in q:
    
    features.append(dfs[i].drop(['product'], axis=1))
    target.append(dfs[i]['product'])

Let's split the data into two samples: train, validation `3 : 1`

In [ ]:
features_train = []
features_valid = []
target_train = []
target_valid = []

for i in q:
    a, b = train_test_split(features[i], test_size=0.25, random_state=12345)
    features_train.append(a)
    features_valid.append(b)
    
    a, b = train_test_split(target[i], test_size=0.25, random_state=12345)
    target_train.append(a)
    target_valid.append(b)
    
    print(features[i].shape)
    print(features_train[i].shape)
    print(features_valid[i].shape)
    print(target_train[i].shape)
    print(target_valid[i].shape)
    print()

Scale features

In [ ]:
numeric = ['f0', 'f1', 'f2']

for i in q:
    scaler = StandardScaler()
    scaler.fit(features_train[i].loc[:, numeric])
    features_train[i].loc[:, numeric] = scaler.transform(features_train[i].loc[:, numeric])
    features_valid[i].loc[:, numeric] = scaler.transform(features_valid[i].loc[:, numeric])
    features[i].loc[:, numeric] = scaler.transform(features[i].loc[:, numeric])

# Train and validate the model

In [ ]:
predicted_valid = []
predicted = []

for i in q:  
    model = LinearRegression()
    model.fit(features_train[i], target_train[i])

    predicted_valid.append(model.predict(features_valid[i]))
    predicted_valid[i] = pd.Series(predicted_valid[i], index=target_valid[i].index) 

    predicted.append(model.predict(features[i]))
    predicted[i] = pd.Series(predicted[i], index=target[i].index) 
    
    rmse = mean_squared_error(target[i], predicted[i]) ** 0.5
    
    print('The average stock of the predicted raw materials of the region', i, '(in thousand bar):', '{:,.2f}'.format(predicted[i].mean()))
    print('RMSE (in thousand bar):', rmse)
    print()

## Conclusion:

The RMSE of the model is quite high, but this is not an indicator. To draw any conclusions, it is also necessary to use other metrics.

# Calculation of the volume of raw materials for break-even development of a new well

In [ ]:
POINTS = 500 # number of studied points
BEST_POINTS = 200 # number of best spots
BUDGET = 10e9 # well development budget, rub.
BUDGET_PER_POINT = BUDGET / BEST_POINTS
BAR_PRICE = 450000 # price per 1 thousand bar.
VOLUME = BUDGET_PER_POINT / BAR_PRICE

print('Required volume for break-even development of a new well, in thousand bar:', '{:,.2f}'.format(VOLUME))

for i in q: print('Average stock in the region', i, '(in thousand bar):', '{:,.2f}'.format(predicted[i].mean()))

## Conclusion

Comparison of the average reserves in the regions with the break-even required volume at first glance shows that there are not enough oil reserves in the selected regions. However, this view may be misleading, as the mean is easily outlier and each case must be studied in detail.

# Function to calculate profit for selected wells and model predictions|

In [ ]:
def revenue(subsample, target):
    index = subsample.sort_values(ascending=False).head(BEST_POINTS).index
    return target[index].sum() * BAR_PRICE - BUDGET_PER_POINT * BEST_POINTS

# Calculation of risks and rewards for each region

In [ ]:
for i in q:
    state = np.random.RandomState(12345)
    values = []
    for j in range(1000):

        subsample = predicted[i].sample(n=POINTS, replace=True, random_state=state)
        values.append(revenue(subsample, target[i]))

    values = pd.Series(values)

    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean_value = values.mean()
    risk = (values[values < 0].count() / values.count()) * 100

    print('Average profit of the region', i, '(rub.)', '{:,.2f}'.format(mean_value))
    print('95% confidence interval (rub.): от','{:,.2f}'.format(lower),'до', '{:,.2f}'.format(upper))
    print('Риск убытков (%):','{:.2f}'.format(risk))
    print()
    sns.distplot(values, kde=True)
    plt.xlabel('Revenue')
    plt.show()

## Conclusion

As a development region, I suggest choosing `"region 1"`, because:
  - The 2.5% quantile of this region has the largest value, which differs many times from other regions
  - the risk of loss is minimal
  - average value of profit - the largest of the presented
 
That is, with a 95% probability we can say that the profit will be at least more than 37 million rubles.

In [ ]:
values = ['a', 'b', 'c', 'd', 'e']
values = pd.Series(values)
state = np.random.RandomState(12345)
for i in range(5):
    
    subsample = values.sample(n=2, replace=True, random_state=12345)
    print(state)
    print(subsample)
    print()